# Basic Transducer, python and go.

In [17]:
# First we connect to database.
%load_ext sql
%sql postgresql://ftian@localhost/ftian

'Connected: ftian@ftian'

The following is a very twisted way of writing
```
select * from 
( select i::int32 as i32, i::float32 as f32, 'Append to ' || i as t
  from generate_series(1, 20) i
) t
```

In [10]:
# Python version.
trpy = """select 
dg_utils.transducer_column_int4(1) as i32,
dg_utils.transducer_column_float4(2) as f32,
dg_utils.transducer_column_text(3) as t,
dg_utils.transducer($PHI$PhiExec python2
# Below is a valid python prorgram.
import vitessedata.phi
vitessedata.phi.DeclareTypes('''
// BEGIN INPUT TYPES
// a int32
// b float32
// c string
// END INPUT TYPES
//
// BEGIN OUTPUT TYPES
// x int32
// y float32
// z string
// END OUTPUT TYPES
''')

def doit():
    while True:
        rec = vitessedata.phi.NextInput()   # Get next input
        if not rec:
            break
            
        if rec[0] % 3 == 1:
            outrec = [None, None, None]
            outrec[0] = rec[0]
            outrec[1] = rec[1] 
            outrec[2] = 'Append to ' + rec[2]
            
            vitessedata.phi.WriteOutput(outrec)
            
    # write a None to signal end
    vitessedata.phi.WriteOutput(None)


if __name__ == '__main__':
    doit()
    
$PHI$), t.*
from (select i::int, i::float4, i::text from generate_series(1, 20) i) t
    
"""

In [18]:
rows = %sql $trpy
print(rows)

7 rows affected.
+-----+------+--------------+------------+------+------+------+
| i32 | f32  |      t       | transducer |  i   | i_1  | i_2  |
+-----+------+--------------+------------+------+------+------+
|  1  | 1.0  | Append to 1  |     -1     | None | None | None |
|  4  | 4.0  | Append to 4  |     -1     | None | None | None |
|  7  | 7.0  | Append to 7  |     -1     | None | None | None |
|  10 | 10.0 | Append to 10 |     -1     | None | None | None |
|  13 | 13.0 | Append to 13 |     -1     | None | None | None |
|  16 | 16.0 | Append to 16 |     -1     | None | None | None |
|  19 | 19.0 | Append to 19 |     -1     | None | None | None |
+-----+------+--------------+------------+------+------+------+


In [14]:
# go version
trgo = """select
dg_utils.transducer_column_int4(1) as i32,
dg_utils.transducer_column_float4(2) as f32,
dg_utils.transducer_column_text(3) as t,
dg_utils.transducer($PHI$PhiExec go
// Below is a valid go prorgram.
// BEGIN INPUT TYPES
// a int32
// b float32
// c string
// END INPUT TYPES
//
// BEGIN OUTPUT TYPES
// x int32
// y float32
// z string
// END OUTPUT TYPES

package main
func main() {
    for rec := NextInput(); rec != nil; rec = NextInput() {
        a, _ := rec.Get_a()
        b, _ := rec.Get_b()
        c, _ := rec.Get_c()
        
        if a % 3 == 1 {
            var outrec OutRecord
            outrec.Set_x(a)
            outrec.Set_y(b)
            outrec.Set_z("Append to " + c)
            WriteOutput(&outrec)
        }
    }
    WriteOutput(nil)
}

$PHI$), t.* 
from (select i::int, i::float4, i::text from generate_series(1, 20) i) t
"""

In [19]:
rows = %sql $trgo
print(rows)

7 rows affected.
+-----+------+--------------+------------+------+------+------+
| i32 | f32  |      t       | transducer |  i   | i_1  | i_2  |
+-----+------+--------------+------------+------+------+------+
|  1  | 1.0  | Append to 1  |     -1     | None | None | None |
|  4  | 4.0  | Append to 4  |     -1     | None | None | None |
|  7  | 7.0  | Append to 7  |     -1     | None | None | None |
|  10 | 10.0 | Append to 10 |     -1     | None | None | None |
|  13 | 13.0 | Append to 13 |     -1     | None | None | None |
|  16 | 16.0 | Append to 16 |     -1     | None | None | None |
|  19 | 19.0 | Append to 19 |     -1     | None | None | None |
+-----+------+--------------+------------+------+------+------+
